In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
embeddings = embed(["cat is on the mat dog is in the fog"])

embeddings

In [ ]:
path = "../Clean/lemma_allresult.csv"
dataset = pd.read_csv(path)

In [ ]:
feat = dataset.iloc[:,0]
target = dataset.iloc[:,1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(feat,target, test_size = 0.2, train_size = 0.8, random_state = 42)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Reshape((20, 1), input_shape=(20,)))
model.add(tf.keras.layers.GRU(32))
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,
          Y_train,
         epochs=10,
         validation_split=0.2,
         verbose=1,
         batch_size=32)

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)

In [ ]:
np.set_printoptions(suppress=True)
result = model.predict(X_test)

result

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=Y_test , y_pred=result>0.5)

cm